In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)
%matplotlib inline

def weight_variable(shape, name):
    return tf.Variable(tf.truncated_normal(shape = shape, stddev = 0.1), name)

def bias_variable(shape, name):
    return tf.Variable(tf.zeros(shape = shape), name)


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
batch_size = 256
g_dim = 100

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 2, 2, 1], padding = 'SAME')
def deconv2d(x, W, output_shape):
    return tf.nn.conv2d_transpose(x, W, output_shape, strides = [1, 2, 2, 1], padding = 'SAME')

x_d=tf.placeholder(tf.float32,shape=[None,784])
x_g=tf.placeholder(tf.float32,shape=[None,g_dim])
weights={
    "w_d1":weight_variable([5,5,1,32],"w_d1"),
    "w_d2":weight_variable([5,5,32,64],"w_d2"),
    "w_d3":weight_variable([7*7*64,1],"w_d3"),
    
    "w_g1" : weight_variable([g_dim, 4 * 4 * 64], "w_g1"),
    "w_g2" : weight_variable([5, 5, 32, 64], "w_g2"),
    "w_g3" : weight_variable([5, 5, 16, 32], "w_g3"),
    "w_g4" : weight_variable([5, 5, 1, 16], "w_g4")
}

biases = {
    "b_d1" : bias_variable([32], "b_d1"),
    "b_d2" : bias_variable([64], "b_d2"),
    "b_d3" : bias_variable([1], "b_d3"),
    "b_g1" : bias_variable([4 * 4 * 64], "b_g1"),
    "b_g2" : bias_variable([32], "b_g2"),
    "b_g3" : bias_variable([16], "b_g3"),
    "b_g4" : bias_variable([1], "b_g4"),
}

var_d = [weights["w_d1"], weights["w_d2"], weights["w_d3"], biases["b_d1"], biases["b_d2"], biases["b_d3"]]
var_g = [weights["w_g1"], weights["w_g2"],weights["w_g3"], weights["w_g4"], biases["b_g1"], biases["b_g2"],biases["b_g3"], biases["b_g4"]]

In [3]:
def generator(z):
    #100x1
    h_g1=tf.nn.relu(tf.add(tf.matmul(z,weights["w_g1"]),biases["b_g1"]))
    #-1x4*4*128
    h_g1_reshape=tf.reshape(h_g1,[-1,4,4,64])
    
    output_shape_g2=tf.stack([tf.shape(z)[0],7,7,32])
    h_g2=tf.nn.relu(tf.add(deconv2d(h_g1_reshape,weights["w_g2"],output_shape_g2),biases["b_g2"]))
    
    output_shape_g3=tf.stack([tf.shape(z)[0],14,14,16])
    h_g3=tf.nn.relu(tf.add(deconv2d(h_g2,weights["w_g3"],output_shape_g3),biases["b_g3"]))
    
    output_shape_g4=tf.stack([tf.shape(z)[0],28,28,1])
    h_g4=tf.nn.tanh(tf.add(deconv2d(h_g3,weights["w_g4"],output_shape_g4),biases["b_g4"]))
    
    return h_g4

def discriminator(x):
    x_reshape=tf.reshape(x,[-1,28,28,1])
    
    h_d1=tf.nn.relu(tf.add(conv2d(x_reshape,weights["w_d1"]),biases["b_d1"]))
    h_d2=tf.nn.relu(tf.add(conv2d(h_d1,weights["w_d2"]),biases["b_d2"]))
    h_d2_reshape=tf.reshape(h_d2,[-1,7*7*64])
    h_d3=tf.nn.sigmoid(tf.add(tf.matmul(h_d2_reshape,weights["w_d3"]),biases["b_d3"]))
    return h_d3


In [4]:
def sample_Z(m,n):
    return np.random.uniform(-1.,1.,size=[m,n])

g_sample=generator(x_g)
d_real=discriminator(x_d)
d_fake=discriminator(g_sample)

d_loss=-tf.reduce_mean(tf.log(d_real)+tf.log(1.-d_fake))
g_loss=-tf.reduce_mean(tf.log(d_fake))

d_optimizer=tf.train.AdamOptimizer(0.0001).minimize(d_loss,var_list=var_d)
g_optimizer=tf.train.AdamOptimizer(0.001).minimize(g_loss,var_list=var_g)

def plot(samples):
    fig=plt.figure(figsize=(4,4))
    gs=gridspec.GridSpec(4,4)
    for i, sample in enumerate(samples):
        ax=plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28,28,),cmap='gray')
        

In [ ]:
sess = tf.InteractiveSession()
init_op = tf.global_variables_initializer()
sess.run(init_op)
for step in range(20001):
    batch_x = mnist.train.next_batch(batch_size)[0]
    _, d_loss_train = sess.run([d_optimizer, d_loss], feed_dict = {x_d: batch_x, x_g: sample_Z(batch_size, g_dim)})
    _, g_loss_train = sess.run([g_optimizer, g_loss], feed_dict = {x_g: sample_Z(batch_size, g_dim)})

    if step <= 1000:
        if step % 100 == 0:
            print("step %d, discriminator loss %.5f" % (step, d_loss_train)),
            print(" generator loss %.5f" % (g_loss_train))
        if step % 1000 == 0: 
            g_sample_plot = g_sample.eval(feed_dict = {x_g: sample_Z(16, g_dim)})
            plot(g_sample_plot)
    else:
        if step % 1000 == 0:
            print("step %d, discriminator loss %.5f" % (step, d_loss_train)),
            print(" generator loss %.5f" % (g_loss_train))
        if step % 2000 == 0: 
            g_sample_plot = g_sample.eval(feed_dict = {x_g: sample_Z(16, g_dim)})
            plot(g_sample_plot)


step 0, discriminator loss 1.57141  generator loss 0.80381
step 100, discriminator loss 2.22205  generator loss 0.44675
step 200, discriminator loss 1.34972  generator loss 1.52780
step 300, discriminator loss 2.12599  generator loss 0.92300
step 400, discriminator loss 2.73131  generator loss 0.42532
step 500, discriminator loss 2.09176  generator loss 0.53778
step 600, discriminator loss 1.91716  generator loss 0.54869
step 700, discriminator loss 1.92316  generator loss 0.55509
step 800, discriminator loss 1.78746  generator loss 0.66446
step 900, discriminator loss 1.36992  generator loss 0.79376
step 1000, discriminator loss 1.06971  generator loss 0.87361
step 2000, discriminator loss 1.12251  generator loss 0.77568
step 3000, discriminator loss 0.68136  generator loss 1.48890
step 4000, discriminator loss 0.09073  generator loss 2.64718
step 5000, discriminator loss 0.04602  generator loss 4.03345
step 6000, discriminator loss 0.00873  generator loss 5.98883
step 7000, discrimin